In [10]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import *
from sklearn.svm import *
from sklearn.tree import *
from sklearn.multiclass import *
from sklearn.dummy import *
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
train_data = np.load('train_data.npz')
validation_data = np.load('validation_data.npz')
test_data = np.load('test_data_public.npz')

In [12]:
def test_classifier(cl, train_x, train_y):
    print ('Train accuracy: ', cl.score(train_x, train_y))
    print ('Cross-validation accuracy: ', cross_val_score(cl, train_x, train_y, cv=10).mean())

In [13]:
train_x = train_data['X'][1950:].copy()
train_y = train_data['y'][1950:].copy()
validation = validation_data['X'].copy()
test = test_data['X'].copy()

In [14]:
const = 0
def binarize(x):
    x[x > const] = 255
    x[x<= const] = 0
    return x

train_x = binarize(train_x)
validation = binarize(validation) # not forgetting about preprocessing validation data
test = binarize(test)

In [15]:
limit = 4
def num(x, y):
    return 28*x+y;

def dfs(number,d, x):
    d.add(number)
    indexes = (-28, -1, 1, 28);
    count = 0
    for j in indexes:
        if (number+j>=0 and number+j<(28*28)):
            if (not(number+j in d) and x[number+j]==255):
                count += dfs(number+j, d, x)
    return (count+1)

def coloring(number, s, x):
    s.add(number)
    indexes = (-28, -1, 1, 28);
    count = 0
    for j in indexes:
        if (number+j>=0 and number+j<(28*28)):
            if (not(number+j in s) and x[number+j]==255):
                count += coloring(number+j, s, x)
    return (count+1)
def work_on(x , lim):
    comp = 0;
    d = set()
    mark = 0
    s =set()
    for i in range(0,28):
        if (mark == 1): break
        for j in range(0,28):
            if (not(num(i,j) in d) and x[num(i,j)]==255):
                d.add(num(i,j))
                t = dfs(num(i,j), d, x)
                #print(t)
                if (t >lim):
                    coloring(num(i, j), s, x)
                    mark = 1;
                    
    for i in range(0,28*28):
        if not(i in s):
            x[i] = 0
        #print(i, x[i])
    
                
def best_comp(x, lim):
    for i in range(0, len(x)):
        work_on(x[i], lim)
#work_on(train_x[60])


In [16]:
limit = 4
def make_rm(x):
    to_rem = set()
    neibours = [-29, -28, -27, -1, 1, 27, 28, 29]
    for i in range(0, 28**2):
        n_num = 0;
        for j in neibours:
            if (i+j<28*28 and i+j >=0):
                if x[i+j] == 255:
                    n_num+=1
        if n_num < limit:
            to_rem.add(i)
    for i in to_rem:
        x[i] = 0
def remove_otr(x):
    for i in range(0, len(x)):
        make_rm(x[i])


In [17]:
best_comp(train_x, 40)
best_comp(validation, 40)
best_comp(test, 40)
remove_otr(train_x)
remove_otr(validation) 
remove_otr(test) 
remove_otr(train_x)
remove_otr(validation) 
remove_otr(test) 
best_comp(train_x, 40)
best_comp(validation, 40)
best_comp(test, 40)

In [18]:
np.save('train950+.npm', train_x)
np.save('validation.npm', validation)
np.save('test.npm', test)

In [19]:
cl =AdaBoostClassifier(
DecisionTreeClassifier(max_depth=8), n_estimators=800, learning_rate=1.5, algorithm="SAMME.R")
cl.fit(train_x, train_y)
test_classifier(cl, train_x, train_y)

Train accuracy:  1.0
Cross-validation accuracy:  0.876315789474


In [20]:
fout = open('answer.csv', 'w')
fout.write('id,answer\n')
data = []
i = 0
for y in cl.predict(test):
    fout.write('%d,%d\n' % (i, y))
    data.append(y)
    i += 1

fout.close()
 